In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import os

def list_files(directory):
    files = []
    for dirname, _, filenames in os.walk(directory):
        for filename in filenames:
            files.append(os.path.join(dirname, filename))
    return files

# Directory path from which you want to list the files
train_dataset = '/kaggle/input/air-pollution-image-dataset-from-india-and-nepal/Air Pollution Image Dataset/Air Pollution Image Dataset/Combined_Dataset/IND_and_NEP'

# Get all files in the input directory
all_files = list_files(train_dataset)

# Output the first five files
for file in all_files[:5]:
    print(file)


/kaggle/input/air-pollution-image-dataset-from-india-and-nepal/Air Pollution Image Dataset/Air Pollution Image Dataset/Combined_Dataset/IND_and_NEP/d_Unhealthy/BRI_Un_2023-02-02- 12.00-11.jpg
/kaggle/input/air-pollution-image-dataset-from-india-and-nepal/Air Pollution Image Dataset/Air Pollution Image Dataset/Combined_Dataset/IND_and_NEP/d_Unhealthy/BIR_UNH_VF_2023-02-02- 12.00-2-97.jpg
/kaggle/input/air-pollution-image-dataset-from-india-and-nepal/Air Pollution Image Dataset/Air Pollution Image Dataset/Combined_Dataset/IND_and_NEP/d_Unhealthy/BIR_UNH_VF_2023-02-02- 12.00-3-20.jpg
/kaggle/input/air-pollution-image-dataset-from-india-and-nepal/Air Pollution Image Dataset/Air Pollution Image Dataset/Combined_Dataset/IND_and_NEP/d_Unhealthy/MH_UH_2023-02-02- 09.30-1-28.jpg
/kaggle/input/air-pollution-image-dataset-from-india-and-nepal/Air Pollution Image Dataset/Air Pollution Image Dataset/Combined_Dataset/IND_and_NEP/d_Unhealthy/MH_UH_2023-02-10-14.30-1-88.jpg


In [3]:
from tensorflow.keras.layers import Input, Lambda , Dense ,Flatten
from tensorflow.keras.models import Model
#from keras.applications.resnet import ResNet50
from tensorflow.keras.applications import VGG16
#from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img ,array_to_img ,img_to_array
from tensorflow.keras.models import Sequential
from glob import glob
import tensorflow as tf
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import random  
from tensorflow import data as tf_data
from tensorflow.keras import layers

In [3]:
DTYPE = tf.float32

In [12]:
#loading the training and testing paths

original_images_dir= '/kaggle/input/air-pollution-image-dataset-from-india-and-nepal/Air Pollution Image Dataset/Air Pollution Image Dataset/Combined_Dataset/IND_and_NEP'
test_df = pd.read_csv ('/kaggle/input/air-pollution-image-dataset-from-india-and-nepal/Dataset_for_AQI_Classification/Dataset_for_AQI_Classification/testing_data.csv')
{ i for i in  test_df['AQI_Class']}

{'a_Good',
 'b_Moderate',
 'c_Unhealthy_for_Sensitive_Groups',
 'd_Unhealthy',
 'e_Very_Unhealthy',
 'f_Severe'}

In [7]:
#creating a directory for the augmented images
save_dir = '/kaggle/working/augmented_images'
os.makedirs(save_dir, exist_ok=True)


In [6]:
augmented_images_dir = '/kaggle/working/augmented_images'

image_dir='/kaggle/input/air-pollution-image-dataset-from-india-and-nepal/Air Pollution Image Dataset/Air Pollution Image Dataset/Combined_Dataset/IND_and_NEP'
'/kaggle/input/air-pollution-image-dataset-from-india-and-nepal/Air Pollution Image Dataset/Air Pollution Image Dataset/Combined_Dataset/IND_and_NEP/a_Good'
directories = [d for d in os.listdir(image_dir) if os.path.isdir(os.path.join(image_dir, d))]
for i in directories:
    if not os.path.exists(augmented_images_dir+'/'+i):
        os.makedirs(augmented_images_dir+'/'+i)

In [7]:
#image augmentation

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

#img = load_img('/kaggle/input/air-pollution-image-dataset-from-india-and-nepal/Air Pollution Image Dataset/Air Pollution Image Dataset/Combined_Dataset/IND_and_NEP') 
image_dir='/kaggle/input/air-pollution-image-dataset-from-india-and-nepal/Air Pollution Image Dataset/Air Pollution Image Dataset/Combined_Dataset/IND_and_NEP'
'/kaggle/input/air-pollution-image-dataset-from-india-and-nepal/Air Pollution Image Dataset/Air Pollution Image Dataset/Combined_Dataset/IND_and_NEP/a_Good'
directories = [d for d in os.listdir(image_dir) if os.path.isdir(os.path.join(image_dir, d))]
for dirr in directories:
    image_files = [os.path.join(image_dir+'/'+dirr+'/', f) for f in os.listdir(image_dir+'/'+dirr+'/') if f.endswith('.jpg')]
    #print(image_files)
    for image in image_files: 
        img = load_img(image) 
        x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
        x = x.reshape((1,) + x.shape)
        i = 0
        for batch in datagen.flow(x, batch_size=1,
            save_to_dir=(augmented_images_dir+'/'+dirr), save_prefix=[j for j in image[::-1].split('/')][0], save_format='jpg'):
            i += 1
            if i > 5:
                break 
                     


In [8]:
#downloading the pre-trained model
IMAGE_SIZE= [224,224]
pre_trained_model= VGG16(input_shape=IMAGE_SIZE + [3],weights= 'imagenet', include_top=False)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [9]:
#dont train the existing weights
for layer in pre_trained_model.layers:
    layer.trainable = False

In [10]:
#number of output classes
folders = glob('/kaggle/working/augmented_images')

In [14]:
#our layers
x = Flatten()(pre_trained_model.output)

In [16]:
prediction = Dense(6, activation='softmax')(x)
#create model object
model= Model(inputs=pre_trained_model.input, outputs=prediction)

In [17]:
#view the structure of the model
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │       150,534 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,865,222 (56.71 MB)

 Trainable params: 150,534 (588.02 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [18]:
# Compile the model
model.compile(
   loss='categorical_crossentropy',
   optimizer ='adam',
   metrics= ['accuracy']
)

In [19]:
#Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
#training the model
train_generator = train_datagen.flow_from_directory('/kaggle/working/augmented_images',
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical'
)
test_generator = test_datagen.flow_from_dataframe(
    test_df,
    directory='/kaggle/input/air-pollution-image-dataset-from-india-and-nepal/Air Pollution Image Dataset/Air Pollution Image Dataset/Combined_Dataset/All_img',
    x_col='Filename',
    y_col='AQI_Class',
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical'
)
#fit the model
r = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)

Found 73420 images belonging to 6 classes.
Found 2448 validated image filenames belonging to 6 classes.
Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1148/1148 ━━━━━━━━━━━━━━━━━━━━ 0s 841ms/step - accuracy: 0.6120 - loss: 1.0071

W0000 00:00:1713265020.475680      86 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1148/1148 ━━━━━━━━━━━━━━━━━━━━ 1011s 876ms/step - accuracy: 0.6120 - loss: 1.0071 - val_accuracy: 0.6961 - val_loss: 0.7927
Epoch 2/10
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 967s 837ms/step - accuracy: 0.6528 - loss: 0.9096 - val_accuracy: 0.6405 - val_loss: 0.8831
Epoch 3/10
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 967s 837ms/step - accuracy: 0.6637 - loss: 0.8924 - val_accuracy: 0.7496 - val_loss: 0.6535
Epoch 4/10
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 964s 834ms/step - accuracy: 0.6772 - loss: 0.8618 - val_accuracy: 0.7194 - val_loss: 0.7131
Epoch 5/10
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 967s 837ms/step - accuracy: 0.6790 - loss: 0.8572 - val_accuracy: 0.7116 - val_loss: 0.7302
Epoch 6/10
1041/1148 ━━━━━━━━━━━━━━━━━━━━ 1:29 834ms/step - accuracy: 0.6911 - loss: 0.8348

In [ ]:
#plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'] ,label='val loss')
plt.legend()
plt.show()
plt.savefig('Lossval_loss')

#plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val accuracy')
plt.legend()
plt.show()
plt.savefig('Accuracyval_accuracy')




In [ ]:
model.save('/kaggle/working/aug.h5')